# Q4 Transfer Learning with Pre-trained CNN 
Use a pre-trained convolutional neural network (CNN) as a feature extractor and fine-
tune a classifier on a subset of the CIFAR-10 dataset. Set random seeds to 42. Follow
the configuration below:
- Load CIFAR-10 and normalize pixel values to [0,1]
- Use only the first 2000 training samples and first 500 test samples
- Load MobileNetV2 from tensorflow.keras.applications, with include top=False and weights=’imagenet’
- Freeze all layers of the pre-trained base
- Add a classifier on top:
- GlobalAveragePooling2D
- Dense layer with 128 neurons, ReLU activation
- Dropout: 0.2
- Output layer: 10 neurons with softmax
- Optimizer: Adam, learning rate = 0.001
- Loss: sparse categorical crossentropy
- epochs = 5, batch size = 32

Q4.1 Report the test accuracy of the model.

### Imports and seed for reproducibility

In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import numpy as np
import random


seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## Loading CIFAR-10 with subsets and adjusting for MobileNet

Why adjust and what is MobileNet?? From [GeeksForGeeks](https://www.geeksforgeeks.org/computer-vision/what-is-mobilenet-v2/)

MobileNet V2 is a powerful and efficient convolutional neural network architecture designed for mobile and embedded vision applications. Developed by Google, MobileNet V2 builds upon the success of its predecessor, MobileNet V1, by introducing several innovative improvements that enhance its performance and efficiency. 
But more importantly for us: MobileNetV2 has an input layer of 224x224 pixels as input, so the model is trained to recognize 224x224 images, that’s the size it was trained on when it learned to recognize features.

In [54]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), # Adjusting image size for MobileNet
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Standard normalization for 3 channels
])

train_full = datasets.CIFAR10('./data', train=True, download=True, transform=transform)
test_full = datasets.CIFAR10('./data', train=False, download=True, transform=transform) 

train_subset = Subset(train_full, range(2000))
test_subset = Subset(test_full, range(500)) 

# split into train and validation sets
train_size = int(0.8 * len(train_subset))
val_size = len(train_subset) - train_size
train_subset, val_subset = torch.utils.data.random_split(train_subset, [train_size, val_size], 
                                                         generator=torch.Generator().manual_seed(seed))

train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=False)

### Load MobileNetV2 base
In the task it specifies using MobileNetV2 from tensorflow.keras.applications. But since i use PyTorch, the pre-trained model is loaded from torchvision.models instead:

In [57]:
# Freeze all convolutional layers
for param in mobilenet.features.parameters():
    param.requires_grad = False

# Replace classifier
num_features = mobilenet.last_channel  # 1280
mobilenet.classifier = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 10)
)

mobilenet = mobilenet.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet.parameters(), lr=0.001)

EPOCHS = 5

for epoch in range(EPOCHS):
    # Training
    mobilenet.train()
    train_loss = 0
    correct = 0
    total = 0
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = mobilenet(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == y).sum().item()
        total += y.size(0)
    train_acc = correct / total

    # Validation
    mobilenet.eval()
    val_loss = 0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            outputs = mobilenet(X)
            loss = criterion(outputs, y)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct_val += (preds == y).sum().item()
            total_val += y.size(0)
    val_acc = correct_val / total_val

    print(f"Epoch {epoch+1}/{EPOCHS}: Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

# Test evaluation
mobilenet.eval()
correct_test = 0
total_test = 0
with torch.no_grad():
    for X, y in test_loader:
        X, y = X.to(device), y.to(device)
        outputs = mobilenet(X)
        _, preds = torch.max(outputs, 1)
        correct_test += (preds == y).sum().item()
        total_test += y.size(0)

test_acc = correct_test / total_test
print(f"Test Accuracy: {test_acc:.4f}")


Epoch 1/5: Train Acc: 0.3581, Val Acc: 0.5175
Epoch 2/5: Train Acc: 0.5950, Val Acc: 0.6425
Epoch 3/5: Train Acc: 0.6694, Val Acc: 0.6800
Epoch 4/5: Train Acc: 0.7200, Val Acc: 0.6900
Epoch 5/5: Train Acc: 0.7344, Val Acc: 0.7025
Test Accuracy: 0.6620


MobileNetV2 already knows how to detect general image features like edges, colors, and textures from ImageNet. By freezing these layers and only training the new classifier, the model reuses this knowledge to recognize images from CIFAR. With a relatively small dataset of 2,000 samples, the train and validation accuracy doe simprove, reaching around 73% train and 70% validation accuracy. The test accuracy is 66%.